<a href="https://colab.research.google.com/github/prateekbps/ML_Projects/blob/master/Transfer_Learning_Cats_%26_Dogs_Classification_90_test_and_train_accuracy_30_July.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
100% 1.06G/1.06G [00:50<00:00, 23.1MB/s]
100% 1.06G/1.06G [00:50<00:00, 22.7MB/s]


In [3]:
import zipfile
zip_file = zipfile.ZipFile('/content/dogs-vs-cats.zip','r')
zip_file.extractall('/content')
zip_file.close()


In [4]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from keras.models import Sequential

In [5]:
# Pre trained model
from keras.applications.vgg16 import VGG16

In [6]:
# Load VGG16 model without the top layers (include_top=False)
# Fine tuning model as we are changing the block 5
# Fine tuning and feature extraction
vgg16_model = VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=(256,256,3),
)

# Make the model trainable initially
vgg16_model.trainable = True

# Variable to track when to start setting layers as trainable
set_trainable = False

# Iterate over the layers of the VGG16 model
for layer in vgg16_model.layers:
    # Find the layer 'block5_conv1'
    if layer.name == 'block5_conv1':
        set_trainable = True  # Start setting layers as trainable
    if set_trainable:
        layer.trainable = True  # Set layers to be trainable
    else:
        layer.trainable = False  # Set layers before 'block5_conv1' as non-trainable

# Print out the name of each layer and whether it is trainable
for layer in vgg16_model.layers:
    print(layer.name, layer.trainable)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
input_layer False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 True
block5_conv2 True
block5_conv3 True
block5_pool True


In [7]:
model = Sequential()
model.add(vgg16_model)
model.add(Flatten())
# change the DNN Part
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 8, 8, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 32768)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       8,388,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,103,809 (88.13 MB)

 Trainable params: 15,468,545 (59.01 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

In [9]:
vgg16_model.trainable = False

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [14]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=0.2, vertical_flip=True,
                                   horizontal_flip=True, zoom_range=0.3, height_shift_range=0.2,
                                   shear_range=0.3)

test_datagen= train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('/content/train', target_size = (256,256),
                                                    batch_size=64, class_mode = 'binary')

validation_generator = train_datagen.flow_from_directory('/content/test', target_size = (256,256),
                                                    batch_size=64, class_mode = 'binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [17]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
history = model.fit(train_generator, validation_data=validation_generator, epochs=10)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


313/313 ━━━━━━━━━━━━━━━━━━━━ 214s 557ms/step - accuracy: 0.6896 - loss: 0.9985 - val_accuracy: 0.9298 - val_loss: 0.2359
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 197s 468ms/step - accuracy: 0.9325 - loss: 0.1833 - val_accuracy: 0.9332 - val_loss: 0.1597
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 213s 504ms/step - accuracy: 0.9549 - loss: 0.1192 - val_accuracy: 0.9402 - val_loss: 0.1409
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 191s 468ms/step - accuracy: 0.9680 - loss: 0.0921 - val_accuracy: 0.9398 - val_loss: 0.1402
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 201s 467ms/step - accuracy: 0.9796 - loss: 0.0663 - val_accuracy: 0.9384 - val_loss: 0.1455
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 202s 467ms/step - accuracy: 0.9828 - loss: 0.0559 - val_accuracy: 0.9352 - val_loss: 0.1644
Epoch 7/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 146s 464ms/step - accuracy: 0.9869 - loss: 0.0420 - val_accuracy: 0.9366 - val_loss: 0.1566
Epoch 8/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 158s 500ms/step - accuracy: 0.9936 - loss: 0.03